In [1]:
%run "ZeroMQ Python Connector".py
%load_ext autoreload
%autoreload
import pandas as pd

In [2]:
#Create connector instance, with particular port for PUSH, PULL and SUB
connector = DWX_ZeroMQ_Connector()

[INIT] Ready to send commands to METATRADER (PUSH): 32768
[INIT] Listening for responses from METATRADER (PULL): 32769
[INIT] Listening for market data from METATRADER (SUB): 32770


In [23]:
connector._DWX_MTX_SEND_MARKETDATA_REQUEST_(_symbol='GBPUSD', _start=(Timestamp.now()-Timedelta(days=178)).strftime('%Y.%m.%d %H:%M:00'),_timeframe=1440,_end=Timestamp.now().strftime('%Y.%m.%d %H:%M:00'))

{'_action': 'DATA', '_data': {'2018.11.27 00:00': 1.27427, '2018.11.28 00:00': 1.28223, '2018.11.29 00:00': 1.27847, '2018.11.30 00:00': 1.27493, '2018.12.03 00:00': 1.27233, '2018.12.04 00:00': 1.27118, '2018.12.05 00:00': 1.27323, '2018.12.06 00:00': 1.27828, '2018.12.07 00:00': 1.27349, '2018.12.10 00:00': 1.25604, '2018.12.11 00:00': 1.24887, '2018.12.12 00:00': 1.26273, '2018.12.13 00:00': 1.26569, '2018.12.14 00:00': 1.25808, '2018.12.17 00:00': 1.26135, '2018.12.18 00:00': 1.26364, '2018.12.19 00:00': 1.26101, '2018.12.20 00:00': 1.26572, '2018.12.21 00:00': 1.26354, '2018.12.24 00:00': 1.26953, '2018.12.26 00:00': 1.26329, '2018.12.27 00:00': 1.26441, '2018.12.28 00:00': 1.26929, '2018.12.31 00:00': 1.27556, '2019.01.02 00:00': 1.26085, '2019.01.03 00:00': 1.26325, '2019.01.04 00:00': 1.27266, '2019.01.07 00:00': 1.27734, '2019.01.08 00:00': 1.27123, '2019.01.09 00:00': 1.27876, '2019.01.10 00:00': 1.27488, '2019.01.11 00:00': 1.28392, '2019.01.14 00:00': 1.28654, '2019.01.15 0

In [ ]:
response = Series(connector._get_response_())
responseData = Series(response["_data"])
print(type(responseData[0]))
order = connector._generate_default_order_dict()
print(order)

In [ ]:
order = connector._generate_default_order_dict()
connector._DWX_MTX_SEND_MARKETDATA_REQUEST_(_symbol='GBPUSD', _start=(Timestamp.now()-Timedelta(minutes=1)).strftime('%Y.%m.%d %H:%M:00'),_timeframe=1,_end=Timestamp.now().strftime('%Y.%m.%d %H:%M:00'))
response = Series(connector._get_response_())
responseData = Series(response["_data"])
order['_symbol'] = 'GBPUSD'
order['_price'] = responseData[0]
connector._DWX_MTX_NEW_TRADE_(order)

In [ ]:
connector._DWX_MTX_GET_ALL_OPEN_TRADES_()

In [26]:
def obtainMarketDataChunk(symbol,startTime, endTime, timeframe=1):
    outputFrame = pd.DataFrame()
    
    if timeframe == 1:
        chunkSizeMinutes = 125
        chunkSizeDays = 0
    elif timeframe == 1440:
        chunkSizeDays = 125
        chunkSizeMinutes = 0
    pdStartTime = Timestamp.strptime(startTime,'%Y.%m.%d %H:%M:00')
    pdEndTime = Timestamp.strptime(endTime,'%Y.%m.%d %H:%M:00')
    timeDelta = pdEndTime - pdStartTime 
    divMod = divmod(timeDelta,Timedelta(minutes=chunkSizeMinutes, days = chunkSizeDays))
    currentStartTime = None
    currentEndTime = None
    response = None
    print(divMod)
    for x in range(divMod[0]):
        print("--------------------------------------------------------------------------------------------------")
        oldResponse = connector._get_response_()
        currentStartTime = pdStartTime + (x*Timedelta(minutes=chunkSizeMinutes, days = chunkSizeDays))
        currentEndTime = pdStartTime + ((x+1)*Timedelta(minutes=chunkSizeMinutes, days = chunkSizeDays))
        connector._DWX_MTX_SEND_MARKETDATA_REQUEST_(_symbol=symbol, _start=currentStartTime.strftime('%Y.%m.%d %H:%M:00'),
                                                    _timeframe=timeframe,_end=currentEndTime.strftime('%Y.%m.%d %H:%M:00'))
        while (connector._get_response_() == oldResponse):
            pass
        
        response = connector._get_response_()
        outputFrame = outputFrame.append(Series(response['_data']).to_frame())
        sleep(0.1)
        print("--------------------------------------------------------------------------------------------------")
        
    
    print("--------------------------------------------------------------------------------------------------")
    connector._DWX_MTX_SEND_MARKETDATA_REQUEST_(_symbol=symbol, _start=currentEndTime.strftime('%Y.%m.%d %H:%M:00'),
                                                _timeframe=timeframe,_end=endTime)
    oldResponse = connector._get_response_()
    while (connector._get_response_() == oldResponse):
            pass
    response = connector._get_response_()
    outputFrame = outputFrame.append(Series(response['_data']).to_frame())
    print("--------------------------------------------------------------------------------------------------")
    return outputFrame.reset_index()


In [27]:
datachunk = obtainMarketDataChunk('GBPUSD', '2018.10.22 10:00:00', '2019.05.23 01:00:00',timeframe=1440)
print(datachunk.to_string())

(1, Timedelta('87 days 15:00:00'))
--------------------------------------------------------------------------------------------------
{'_action': 'DATA', '_data': {'2018.10.23 00:00': 1.2982, '2018.10.24 00:00': 1.28815, '2018.10.25 00:00': 1.28155, '2018.10.26 00:00': 1.28281, '2018.10.29 00:00': 1.27929, '2018.10.30 00:00': 1.27066, '2018.10.31 00:00': 1.2765, '2018.11.01 00:00': 1.30002, '2018.11.02 00:00': 1.29688, '2018.11.05 00:00': 1.30417, '2018.11.06 00:00': 1.30997, '2018.11.07 00:00': 1.31276, '2018.11.08 00:00': 1.30602, '2018.11.09 00:00': 1.29736, '2018.11.12 00:00': 1.28493, '2018.11.13 00:00': 1.29672, '2018.11.14 00:00': 1.29838, '2018.11.15 00:00': 1.27704, '2018.11.16 00:00': 1.2827, '2018.11.19 00:00': 1.28482, '2018.11.20 00:00': 1.27865, '2018.11.21 00:00': 1.27758, '2018.11.22 00:00': 1.28743, '2018.11.23 00:00': 1.28012, '2018.11.26 00:00': 1.28073, '2018.11.27 00:00': 1.27427, '2018.11.28 00:00': 1.28223, '2018.11.29 00:00': 1.27847, '2018.11.30 00:00': 1.27493